In [ ]:
using JuMP
using Ipopt
using LinearSolve

In [ ]:
hailstones = readlines("../inputs/24") .|> (line ->
     split(line, '@') .|> segment -> split(segment, ',') .|> x -> parse(Float64, x)
    )
hailstones[1]

## Part 1

In [ ]:
hailstones2d = [ [hailstone[1][1:2], hailstone[2][1:2]] for hailstone in hailstones ]
hailstones2d[1]

In [ ]:
bound_min = 200000000000000
bound_max = 400000000000000

area_min = [bound_min, bound_min]
area_max = [bound_max, bound_max]
acc = 0

for (i, lhs) in hailstones2d |> enumerate
    for rhs in hailstones2d[i+1:end]
        intersection_time = LinearProblem(hcat(lhs[2], -rhs[2]), rhs[1] - lhs[1]) |> solve
        if (intersection_time .> 0) |> prod
            intersection_point = lhs[1] + lhs[2] * intersection_time[1]
            if ((intersection_point - area_min .> 0) |> prod) &&
                ((intersection_point - area_max .< 0) |> prod)
                acc += 1
            end
        end
    end
end

### Part 1 solution

In [ ]:
acc

## Part 2

In [ ]:
hailstones = hailstones[1:3]

In [ ]:
model = Model(Ipopt.Optimizer)
set_silent(model)
positions = hailstones[:] .|> hailstone -> hailstone[1]
velocities = hailstones[:] .|> hailstone -> hailstone[2]

@variables(model, begin
        s_x
        s_y
        s_z
        v_x
        v_y
        v_z
        t[1:length(hailstones)] 
    end)

for i in 1:length(hailstones)
    @constraints(model, begin
            s_x + v_x * t[i] == hailstones[i][1][1] + hailstones[i][2][1] * t[i]
            s_y + v_y * t[i] == hailstones[i][1][2] + hailstones[i][2][2] * t[i]
            s_z + v_z * t[i] == hailstones[i][1][3] + hailstones[i][2][3] * t[i]
        end)
end

model

In [ ]:
optimize!(model);

In [ ]:
values = value.((s_x, s_y, s_z, v_x, v_y, v_z, t[:]...)) .|> value -> round(Int, value)

### Part 2 solution

In [ ]:
values[1:3] |> sum